In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [2]:
df = pd.read_csv('ocr_rows_for_labeling.csv', sep=";")
df.head()

,image,row_idx,row_ratio,type,text,label
0,X00016469612.jpg,0,0.000,header,tan woon yann,0
1,X00016469612.jpg,1,0.038,header,BOOK TA.K (TAMAN DAYA) SDN BHD,0
2,X00016469612.jpg,2,0.077,header,789417-W,0
3,X00016469612.jpg,3,0.115,header,"NO.5: 55,57 & 59, JALAN SAGU 18,",0
4,X00016469612.jpg,4,0.154,header,"TAMAN DAYA,",0


In [3]:
df['label'].value_counts()

label
0    604
1     57
Name: count, dtype: int64

In [4]:
df["alpha_ratio"] = df["text"].apply(
    lambda s: sum(c.isalpha() for c in s) / max(len(s), 1)
)

In [5]:
df["has_digit"] = df["text"].apply(
    lambda s: int(any(c.isdigit() for c in s))
)

In [6]:
df.head(10)

,image,row_idx,row_ratio,type,text,label,alpha_ratio,has_digit
0,X00016469612.jpg,0,0.000,header,tan woon yann,0,0.846154,0
1,X00016469612.jpg,1,0.038,header,BOOK TA.K (TAMAN DAYA) SDN BHD,0,0.733333,0
2,X00016469612.jpg,2,0.077,header,789417-W,0,0.125000,1
3,X00016469612.jpg,3,0.115,header,"NO.5: 55,57 & 59, JALAN SAGU 18,",0,0.343750,1
4,X00016469612.jpg,4,0.154,header,"TAMAN DAYA,",0,0.818182,0
5,X00016469612.jpg,5,0.192,header,"81100 JOHOR BAHRU,",0,0.555556,1
6,X00016469612.jpg,6,0.231,header,JOHOR.,0,0.833333,0
7,X00016469612.jpg,7,0.269,header,Document No :TD01167104,0,0.521739,1
8,X00016469612.jpg,8,0.308,header,Date : 25/12/2018 8:13:39 PM,0,0.214286,1
9,X00016469612.jpg,9,0.346,summary,Cashier: MANIS,0,0.857143,0


In [8]:
FEATURES = ["row_ratio", "alpha_ratio", "has_digit"]

X = df[FEATURES]
y = df["label"]

In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42)

print("Train:", y_train.value_counts(normalize=True))
print("Val:", y_val.value_counts(normalize=True))
print("Test:", y_test.value_counts(normalize=True))

Train: label
0    0.91342
1    0.08658
Name: proportion, dtype: float64
Val: label
0    0.919192
1    0.080808
Name: proportion, dtype: float64
Test: label
0    0.91
1    0.09
Name: proportion, dtype: float64
